# Fast.AI
Notes and example code on using fast.ai.

Best place to find information is in the docs!
https://docs.fast.ai/

FastAI can deal with:
- images
- text
- tabular data


# Chapter 1 - Intro

In [1]:
from fastai.vision.all import *
from fastai.text.all import *
from fastai.tabular.all import *


In [ ]:
# Cat/Dog example

path = untar_data(URLs.PETS)/'images'

# Callback to deal with the incoming pictures of cats and dogs
# Files that start with an upper-case letter are pictures of cats
def is_cat(x): 
    return x[0].isupper()

# Dataloader - important bit is the valid_pct - which tells fastai to hold 20% for validation (not included in training)
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

# Create CNN using resnet34 model
# The 34 in resnet34 refers to the number of layers in this variant of the architecture (other options are 18, 50, 101, and 152). 
# A metric is a function that measures the quality of the model's predictions using the validation set, and will be printed at 
# the end of each epoch. In this case, we're using error_rate, which is a function provided by fastai that does just what it 
# says: tells you what percentage of images in the validation set are being classified incorrectly. Another common metric for 
# classification is accuracy (which is just 1.0 - error_rate). fastai provides many more,
learn = vision_learner(dls, resnet34, metrics=error_rate)

# Fine tune using the dataloader object
learn.fine_tune(1)

# Supply image to test with
uploader = SimpleNamespace(data = ['images/chapter1_cat_example.jpg'])

img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")


In [ ]:
# Finetune the AWD_LSTM model on IMDB data to be able to calculate setiment of a string.
# Struggle to run this one locally due to running out of VRAM.  Reduce the bs (batchsize) below.

# grab the IMDB data
# bs is the batch-size - which can be reduced to 8, 4, etc to reduce memory usage during training
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', bs=32)
# create the AWS_LSTM model
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
# fine tune
learn.fine_tune(4, 1e-2)

# example useage:
learn.predict("I really liked that movie!")
# -> ('neg', tensor(0), tensor([0.8786, 0.1214]))

In [2]:
# Tabular data example

path = untar_data(URLs.ADULT_SAMPLE)

dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
cat_names = ['workclass', 'education', 'marital-status', 'occupation',
                'relationship', 'race'],
cont_names = ['age', 'fnlwgt', 'education-num'],
procs = [Categorify, FillMissing, Normalize])

learn = tabular_learner(dls, metrics=accuracy)    

# there's no pretrained model for tabular data that we finetune, so we train from-scratch
# run 3 epochs
learn.fit_one_cycle(3)

# use this to show the model
learn.show_results()

# TODO: need to figure out how to run .predict() against this model...?

/mnt/secondary/hack/fastbook-course/.venv/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


epoch,train_loss,valid_loss,accuracy,time
0,0.380413,0.382440,0.818489,00:04
1,0.361306,0.360611,0.828931,00:02
2,0.349509,0.358183,0.830006,00:02


,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,16.0,3.0,9.0,6.0,5.0,1.0,0.105563,1.080978,-0.032015,0.0,0.0
1,5.0,12.0,3.0,4.0,4.0,5.0,1.0,-0.114287,0.053980,-0.422627,0.0,0.0
2,5.0,5.0,3.0,15.0,1.0,5.0,1.0,0.691827,1.196191,-2.766299,0.0,0.0
3,5.0,15.0,3.0,11.0,1.0,5.0,1.0,-0.041004,1.512582,1.921046,0.0,1.0
4,5.0,12.0,5.0,7.0,2.0,5.0,1.0,-1.066966,-1.508145,-0.422627,0.0,0.0
5,5.0,12.0,5.0,4.0,4.0,5.0,1.0,-0.993683,0.800836,-0.422627,0.0,0.0
6,5.0,12.0,3.0,8.0,1.0,5.0,1.0,0.545261,0.703742,-0.422627,0.0,0.0
7,3.0,12.0,3.0,12.0,1.0,2.0,1.0,-0.041004,-1.112305,-0.422627,1.0,0.0
8,8.0,11.0,3.0,11.0,1.0,3.0,1.0,0.765110,-0.527510,2.311658,1.0,1.0


## Hyperparameters

Hyperparameters are parameters that are set prior to the training process and remain constant during training. These parameters control the behavior of the algorithm and influence the performance of the model. Unlike model parameters, which are learned during the training process (such as weights in neural networks), hyperparameters are set by the practitioner or determined through techniques like hyperparameter tuning.

Here are some common hyperparameters in deep learning:

1. Learning Rate: This hyperparameter controls the step size during the optimization process. A higher learning rate can lead to faster convergence but may overshoot the optimal solution, while a lower learning rate may converge slowly.

2. Batch Size: It determines the number of samples that are propagated through the network at once. Larger batch sizes can lead to faster training but require more memory, while smaller batch sizes may lead to slower convergence but can provide a more accurate estimate of the gradient.

3. Number of Epochs: An epoch is one complete pass through the entire training dataset. The number of epochs is a hyperparameter that specifies how many times the learning algorithm will work through the entire training dataset.

4. Network Architecture: Hyperparameters related to the architecture of the neural network, such as the number of layers, the number of neurons in each layer, types of activation functions, etc.

5. Regularization Parameters: Regularization is a technique used to prevent overfitting by adding a penalty term to the loss function. Hyperparameters such as L1 or L2 regularization strength are examples.

6. Dropout Rate: Dropout is a regularization technique where randomly selected neurons are ignored during training. The dropout rate is the probability that a neuron will be dropped out during training.

7. Optimizer Parameters: Hyperparameters related to the optimization algorithm used during training, such as momentum, decay rates, etc.

8. Activation Functions: Hyperparameters related to the choice of activation functions used in the hidden layers of a neural network, such as ReLU, sigmoid, tanh, etc.

Choosing appropriate hyperparameters is crucial for achieving good performance and generalization of the model. Hyperparameter tuning, often done through techniques like grid search or random search, involves experimenting with different combinations of hyperparameters to find the set that yields the best performance on a validation dataset.